In [1]:
import $ivy.`com.chuusai::shapeless:2.3.2`

import $ivy.$                             

### Preamble

Buckle up, it's shapeless time!

In [2]:
import shapeless._

import shapeless._

Let's say we have a case class with just a single string field called `Id`

In [3]:
case class AnId(Id: String)

defined class AnId

This is isomorphic to all sorts of things, including:

* `String`
* `String :: HNil`
* `String with shapeless.labelled.KeyTag[Symbol with shapeless.tag.Tagged[String("Id")],String]`
* `String with shapeless.labelled.KeyTag[Symbol with shapeless.tag.Tagged[String("Id")],String] :: HNil`

We want that second one, which is the kind of representation that shapeless would call a `Generic`

In [4]:
val lg = Generic[AnId]

lg: Generic[AnId]{type Repr = shapeless.::[String,shapeless.HNil]} = $sess.cmd3Wrapper$Helper$anon$macro$2$1@609aaf93

Which lets us do this sort of thing:

In [5]:
val anId = AnId("MyId")
val hlist = lg.to(anId)
val roundTrip = lg.from(hlist)

anId: AnId = AnId("MyId")
hlist: lg.Repr = MyId :: HNil
roundTrip: AnId = AnId("MyId")

And we can do this for anything that we can get a `Generic` for

So next let's write a function that lets us add a string to our class

In [6]:
def addString(id: AnId, s: String)(implicit ev: Generic.Aux[AnId, String :: HNil]) : AnId = {
  val idString :: HNil = ev.to(id)
  val appended = idString + s
  val inHlist = appended :: HNil
  ev.from(inHlist)
}

defined function addString

In [7]:
addString(anId, "bob")

res6: AnId = AnId("MyIdbob")

We have to do a bit of hackery to tell the compiler that we want a `Generic` instance that lets us go between `AnId` and `String :: HNil` - that's what the `Generic.Aux[AnId, String :: HNil]` stuff is, and it's an example of the `Aux Pattern` that's very common in shapeless code

Armed with that, we can now write something that works in general for anything that looks like an `Id`

In [8]:
def addString2[T](id: T, s: String)(implicit ev: Generic.Aux[T, String :: HNil]) : T = {
  val idString :: HNil = ev.to(id)
  val appended = idString + s
  val inHlist = appended :: HNil
  ev.from(inHlist)
}

defined function addString2

In [9]:
addString2(anId, "bob")

res8: AnId = AnId("MyIdbob")

In [10]:
case class AnotherId(id:String)
val anotherId = AnotherId("foo")
addString2(anotherId, "bar")

defined class AnotherId
anotherId: AnotherId = AnotherId("foo")
res9_2: AnotherId = AnotherId("foobar")

Sadly, this implementation is probably *too* general - it's going to kick in for any case class wrapper around a string, and very often the entire reason to wrap strings in case classes is to escape the horror of string addtion.

So let's add a constraint to stop it working for things that aren't `Id`s

In [11]:
trait Id
def addString3[T <: Id](id: T, s: String)(implicit ev: Generic.Aux[T, String :: HNil]) : T = {
  val idString :: HNil = ev.to(id)
  val appended = idString + s
  val inHlist = appended :: HNil
  ev.from(inHlist)
}

defined trait Id
defined function addString3

In [12]:
case class AnotherId2(id:String) extends Id
val anotherId2 = AnotherId2("woop")
addString3(anotherId2, "boop")

defined class AnotherId2
anotherId2: AnotherId2 = AnotherId2("woop")
res11_2: AnotherId2 = AnotherId2("woopboop")

But if we try it for something that's not an `Id`...

In [13]:
case class NotAnId(id:String)

defined class NotAnId

In [13]:
val notAnId = NotAnId("not an Id, apparently")
addString3(notAnId, "boop")

cmd13.sc:2: inferred type arguments [cmd13Wrapper.this.cmd12.wrapper.NotAnId] do not conform to method addString3's type parameter bounds [T <: cmd13Wrapper.this.cmd10.wrapper.Id]
val res13_1 = addString3(notAnId, "boop")
              ^cmd13.sc:2: type mismatch;
 found   : cmd13Wrapper.this.cmd12.wrapper.NotAnId
 required: T
val res13_1 = addString3(notAnId, "boop")
                         ^cmd13.sc:2: could not find implicit value for parameter ev: shapeless.Generic.Aux[T,shapeless.::[String,shapeless.HNil]]
val res13_1 = addString3(notAnId, "boop")
                        ^

: 

Which is nice!

So now we just want some ceremony to let us write it as `id op string`

In [14]:
implicit class IdOps[T <: Id](t: T){
  def /(s:String)(implicit ev: Generic.Aux[T, String :: HNil]) : T = {
    val str :: HNil = ev.to(t)
    ev.from((str + s) :: HNil)
  }
}

defined class IdOps

In [15]:
AnotherId2("Hi, my name is ") / "... what" / "... who"

res14: AnotherId2 = AnotherId2("Hi, my name is ... what... who")

However...

In [15]:
NotAnId("Once upon a time") / ", in a far away land"

cmd15.sc:1: value / is not a member of cmd15Wrapper.this.cmd12.wrapper.NotAnId
val res15 = NotAnId("Once upon a time") / ", in a far away land"
                                        ^

: 

PS. I couldn't bring myself to allow concatenating them with `+` and `Any2StringAdd` is the single worst thing in the entire scala language and I just can't contribute to more of that madness!

Another useful thing to know is that if you name the string inside the case class `toString` then when you print the Id it comes out as just the wrapped string, but you need to add an `override val` to it because **reasons**

In [16]:
case class AwesomeId(override val toString: String) extends Id
val myAwesomeId = AwesomeId("awesome!")
val moreAwesome = myAwesomeId / "awesomer!"

defined class AwesomeId
myAwesomeId: AwesomeId = awesome!
moreAwesome: AwesomeId = awesome!awesomer!